In [1]:
!pip install litellm --quiet
!pip install deep-translator --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.2 MB/s eta 0:00:00


In [2]:
gpt4 = True

m = "gpt-4"
if not gpt4:
  m = "gpt-3.5-turbo"

#Set up

In [3]:
from litellm import completion
from deep_translator import GoogleTranslator
import os

## set ENV variables
os.environ["OPENAI_API_KEY"] = ""

In [4]:
def answer(q):
  messages = [{"content": q,"role": "user"}]
  response = completion(model=m, messages=messages)
  return response.choices[0].message.content

In [5]:
truth = {
    True: {
        "en":"Yes.",
        "be":"Так.",
        "ru":"Да.",
        "uk":"Так."
    },
    False: {
        "en":"No.",
        "be":"Не.",
        "ru":"Нет.",
        "uk":"Ні."
    }
}
truth[True]['uk']

'Так.'

In [6]:
def translate(text, from_ ="uk", to="en"):
    to_translate = text
    translated = GoogleTranslator(source=from_, target=to).translate(to_translate)
    return translated

In [7]:
translate("ти чи він", from_ ="uk", to="en")

'you or him'

# Response generation

In [8]:
from nltk import CFG
from nltk.parse.generate import generate

grammar = CFG.fromstring("""
  AndP -> AndPX | AndPX " and " AndPX
  AndPX -> " N " | AndP " and " AndP
 """)
#With this we "create" all the possible combinations
grammar.productions()

#Here you can see all the productions (sentences) with 5 words
#created with this grammar
possible_sequences = []
i = 0
for production in generate(grammar, depth = 5):
  possible_sequences.append(''.join(production))
  i += 1
possible_sequences = list(set(possible_sequences))
for p in possible_sequences:
  print(p)

 N  and  N  and  N  and  N  and  N 
 N  and  N  and  N 
 N 
 N  and  N  and  N  and  N 
 N  and  N  and  N  and  N  and  N  and  N  and  N  and  N 
 N  and  N  and  N  and  N  and  N  and  N 
 N  and  N  and  N  and  N  and  N  and  N  and  N 
 N  and  N 


In [122]:
grammar = CFG.fromstring("""
OrP -> "(" AndP ")" | "(" AndP " or " AndP ")"  | "(" AndP " or " OrPX ")"
OrPX -> "(" OrP  ")"  | "(" OrP " or " OrP ")"
AndP -> "(" AndPX " and " AndPX ")"  | "(" AndPX ")"
AndPX -> "(N and N)" | "(N)" | "(" AndP ")"  | "(N but not N)" | "(not N)" | "(not N but N)"
 """)
#With this we "create" all the possible combinations
grammar.productions()

#Here you can see all the productions (sentences) with 5 words
#created with this grammar
possible_sequences = []
i = 0
for production in generate(grammar, depth = 5):
  possible_sequences.append(''.join(production))
  i += 1
possible_sequences = list(set(possible_sequences))
# for p in possible_sequences:
#   print(p)
print(possible_sequences)

['(((not N) and (N)) or ((N and N) and (not N)))', '(((not N) and (not N but N)) or ((not N but N) and (N but not N)))', '(((not N but N) and (not N but N)) or ((not N but N)))', '(((not N but N) and (N but not N)) or ((not N) and (not N)))', '(((N but not N)) or ((N) and (N but not N)))', '(((N but not N) and (not N)) or ((N and N) and (N)))', '(((N and N) and (N and N)) or ((not N but N) and (N and N)))', '(((not N but N) and (N and N)) or ((not N but N) and (N and N)))', '(((N but not N) and (not N but N)))', '(((N and N) and (not N but N)) or ((not N) and (N but not N)))', '(((not N but N) and (N)) or ((N) and (N)))', '(((not N) and (not N)) or ((not N) and (not N but N)))', '(((N) and (N but not N)) or ((not N) and (not N but N)))', '(((N) and (N and N)) or ((N) and (N but not N)))', '(((N) and (not N)) or ((not N but N) and (not N)))', '(((N) and (not N but N)) or ((N and N) and (not N)))', '(((N and N) and (not N but N)) or ((not N but N)))', '(((not N) and (not N but N)) or ((N

#Boolean

In [11]:
def par1lev(s):
  op = "("
  cl = ")"
  closure = -1
  res_indicies = [0]
  for i in range(len(s)):
    if s[i] == op:
      closure += 1
      if closure == -1:
        res_indicies.append(i)
    if s[i] == cl:
      closure -= 1
      if closure == -1:
        res_indicies.append(i)
        if i+1!=len(s):
          res_indicies.append(i+1)
  return res_indicies

In [113]:
def evaluate_simple_expression(expr, values):
    # Evaluates a simple boolean expression (without nested parentheses)
    for var, val in values.items():
        expr = expr.replace(var, str(val))
    return eval(expr)

def parse_b(expr, values):
    indices = par1lev(expr)
    if len(indices) <= 2:
        return evaluate_simple_expression(expr, values)

    # Recursively process each subexpression
    while len(indices) > 2:
        start = indices[1]
        end = indices[2]
        sub_expr = expr[start+1:end]
        sub_val = parse_b(sub_expr, values)
        expr = expr[:start] + str(sub_val) + expr[end+1:]
        indices = par1lev(expr)

    return evaluate_simple_expression(expr, values)

# Example usage
expr = "((apple or (bag and cat)) and (not bag and not cat))"
values = {'apple': True, 'bag': False, 'cat':False}
result = parse_b(expr, values)
print(result)  # Expected output: True

True


In [13]:
def make_plural_final(noun):
    """
    This function converts a singular noun to its plural form with further corrections.
    """
    # Nouns ending in -y preceded by a consonant, change the -y into -ies
    if noun[-1] == "y" and noun[-2] not in "aeiou":
        return noun[:-1] + "ies"
    # Nouns ending in -s, -ss, -sh, -ch, -x, or -z, add -es
    elif noun[-1] in "sx" or noun[-2:] in ["ss", "sh", "ch"]:
        return noun + "es"
    # Special case for nouns ending in -z
    elif noun[-1] == "z":
        return noun + "zes"
    # For most other nouns, simply add -s
    else:
        return noun + "s"

# Testing the final corrected function
test_nouns_final = ["cat", "dog", "bus", "wish", "fox", "quiz", "apple", "city",
    "tree", "car", "house", "boat", "flower", "computer", "phone", "river",
    "chair", "book", "pen", "desk", "chair", "school", "dog", "cat",
    "bird", "fish", "ball", "game", "road", "window", "coat"
]

In [106]:
#type 1: conjunction
import random
import math

def gen_conj(nouns, max=10):
  n = random.randint(2, max)

  truth = [0, 1]
  truth_bias = [1, math.exp(max)/1000]
  truth_list = [bool( random.choices(truth, weights=truth_bias, k=1)[0]) for i in range(n)]
  noun_list = [nouns[random.randint(0, max-1)] for i in range(n)]
  print(dict(zip(noun_list, truth_list)))

In [112]:
gen_conj(test_nouns_final)

{'car': True, 'tree': False, 'dog': True, 'fox': True, 'wish': True}


# Test

In [16]:
q = \
"""
Andriy says: "buy a candy and a cookie or a lollipop but not candy."
Boris bought a cookie and a lollipop.
Did Boris's purchase fully meet Andriy's requirements? Say briefly: "yes" or "no"
"""
answer(q)

'Yes'

In [17]:
answer(translate(q, from_ = "en", to="de"))

'Ja'